<a href="https://colab.research.google.com/github/saraHuang/LLM_study/blob/main/LangChain_%E7%AF%87_%E6%89%8B%E6%8A%8A%E6%89%8B%E5%BB%BA%E7%AB%8B_Data_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🥱 LangChain 篇_手把手建立 Data Agent

❤️ Created by [hengshiousheu](https://huggingface.co/Heng666).

# 整體概念

本筆記本用於交售第一次接觸大型語言模型開發者，如何透過 LangChain 工具來建構自己的 LLM 應用程式開發。本範例終將提供以下以幾個使用範例：

1. `建構 Text2SQL`: 基於使用者自然語言轉換成 SQL 用語
2. `查詢 SQL 資料庫`: 建構專屬 Chain 達成自然語言直接操作資料庫
3. `與 SQLDB Agent 互動`: 使用 Agner 進行嗆大且具備彈性的操作。

![sql_usecase.png](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/sql_usecase.png?raw=1)圖片取自: LangChain SQL_usecase 官網

# 使用案例說明

通常，企業資料通是存放在 SQL 資料庫中

大型語言模型的出現，讓使用自然語言與 SQL 資料庫互動成為可能。

其中，LangChain 提供 SQL 鏈和代理，以基於自然語言提示建立和執行 SQL 查詢。

這些與 SQLAlchemy 支援的任何 SQL 直接相容（例如，MySQL、PostgreSQL、Oracle SQL、Databricks、SQLite）。

LangChain 可用於以下使用案例：
- 根據自然語言問題產生將執行的查詢
- 建立可根據資料庫資料回答問題的聊天機器人



# 快速入門

## 建置環境

首先，取得必要的套件並設定環境變數：

In [ ]:
%pip install --upgrade --quiet langchain langchain-experimental langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00


我們需要設置 環境變數 `OPENAI_API_KEY` ，可以直接完成，如下所示：

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## 如果是採用 Gemini 的話，可以修正成為以下內容

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

In [ ]:
!pip install --quiet langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

# 設置 Gemini API Key
os.environ["GOOGLE_API_KEY"] = "你的_Gemini_API_Key"

# 初始化 Gemini 模型
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # 或其他 Gemini 模型名稱，例如 gemini-2.0-flash, gemini-1.5-flash
    temperature=0
)

##建置資料庫

以下範例將使用與 Chinook 資料庫的 SQLite 連線。

依照安裝步驟，在與這個筆記本相同的目錄中建立 Chinook.db ：

* `安裝 sqlite3` 套件於環境中
* `使用 sqlite3 指令` 讀取內建 DB

現在， Chinhook.db 在我們的目錄中。

讓我們建立一個 SQLDatabaseChain 來建立和執行 SQL 查詢。

要是不確定的話，可以看 [SQL Tutorial](https://www.sqlitetutorial.net/sqlite-sample-database/)來了解指令操作，還有資料庫中都裝了什麼

In [ ]:
!apt-get install --upgrade --quiet sqlite3
!wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql
!sqlite3 Chinook.db '.read Chinook_Sqlite.sql'

Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 769 kB of archives.
After this operation, 1,873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.4 [769 kB]
Fetched 769 kB in 0s (3,303 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.4_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.4) ...
Setting up sqlite3 (3.37.2-2ubuntu0.4) ...
Processing triggers for man-db (2.10.2-1) ...
--2025-06-24 04:59:46--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 1

## 建立語言模型 pipe

這個環節中，我先指定語言模型為 OPEANAI model
接著建立 SQLDBChain 來進行操作。

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import OpenAI

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

接著可以使用 Chain 透過語言模型來跟資料庫進行對話

In [ ]:
query_response = db_chain.run("How many employes inside?")
print(query_response)



> Entering new SQLDatabaseChain chain...
How many employes inside?
SQLQuery:SELECT COUNT(*) FROM Employee
SQLResult: [(8,)]
Answer:There are 8 employees in the database.
> Finished chain.
There are 8 employees in the database.



### 想了解更多？

好奇知道還有哪些表格行資料可以讀取的話，可以參考 Langchain 當中的文件，例如

- [讀取 CSV 檔案](https://python.langchain.com/docs/integrations/toolkits/csv)
- [讀取 Pandas DAtaFrame](https://python.langchain.com/docs/integrations/toolkits/pandas)

## 案例ㄧ: Text-to-SQL


In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

讓我們建立將建構 SQL 查詢的鏈：


In [ ]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM "Employee"


在根據使用者問題建構 SQL 查詢後，我們可以執行查詢：

In [ ]:
db.run(response)

'[(8,)]'

正如我們所見，SQL 查詢建構器鏈只*建立查詢*，而我們分別處理查詢執行。

### 建立 Prompt

In [ ]:
from langchain.prompts import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}.

Some examples of SQL queries that correspond to questions are:

{few_shot_examples}

Question: {input}"""

CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "few_shot_examples", "table_info", "dialect"],
    template=TEMPLATE,
)

我們也可以在 LangChain 提示中心訪問此提示。

In [ ]:
%pip install --upgrade --quiet langchainhub

In [ ]:
from langchain import hub

CUSTOM_PROMPT = hub.pull("rlm/text-to-sql")

## 案例二: Text-to-SQL 查詢與執行

我們可以從 `langchain_experimental` 建立 `SQLDatabaseChain`  並執行 SQL 查詢。

In [ ]:
from langchain_experimental.sql import SQLDatabaseChain
from langchain_openai import OpenAI

llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM Employee
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

正如我們所見，我們獲得與前一個案例相同的結果。

在這裡，鏈也處理查詢執行，並根據使用者問題和查詢結果提供最終答案。

**使用此方法時要小心**，因為它容易受到 `SQL Injection` 的影響：

* 鏈正在執行由 LLM 建立的查詢，並且未經過驗證
* 例如，記錄可能會無意中建立、修改或刪除_



這就是我們看到 `SQLDatabaseChain` 在 `langchain_experimental` 內部的原因。


### 嘗試更多？

可以通過多種方式增強 SQLDatabaseChain 的性能：

* Adding sample rows 添加範例列
* 指定自訂表格資訊
*  使用 Query Checker 自行更正使用參數 use_query_checker=True 的無效 SQL
* 自訂 LLM 提示包含特定說明或相關資訊，使用參數 prompt=CUSTOM_PROMPT
* 取得中間步驟使用參數 return_intermediate_steps=True 存取 SQL 陳述式以及最終結果
* 限制查詢將傳回的行數，使用參數 top_k=5

在資料庫中的表格數目很大的情況下，您可能會發現 SQLDatabaseSequentialChain 很實用。
這個 Sequential Chain 處理以下程序：

1. 根據使用者問題決定要使用哪些表格
2. 僅使用相關表格呼叫一般 SQL 資料庫鏈

In [ ]:
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=[
        "Track"
    ],  # we include only one table to save tokens in the prompt :)
    sample_rows_in_table_info=2,
)

範例資料列會在每個對應表格的欄位資訊之後加入 prompt 中。

我們可以使用 `db.table_info` 並檢查哪些範例資料列包含在內：

In [ ]:
print(db.table_info)


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
2 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Angus Young, Malcolm Young, Brian Johnson	343719	11170334	0.99
2	Balls to the Wall	2	2	1	U. Dirkschneider, W. Hoffmann, H. Frank, P. Baltes, S. Kaufmann, G. Hoffmann	342562	5510424	0.99
*/


## 案例三: SQL agents

LangChain 有一個 SQL Agent，它提供比 `SQLDatabaseChain`更靈活的方式與 SQL 資料庫互動。

使用 SQL Agent 的主要優點是：

它可以根據資料庫的結構以及資料庫的內容（例如描述特定表格）來回答問題

它可以透過執行產生的查詢、擷取追蹤並正確地重新產生查詢，來從錯誤中復原

要初始化代理，我們使用 `create_sql_agent` 函數。

此 Agent 包含 `SQLDatabaseToolkit` ，其中包含下列工具：

* 建立並執行查詢
* 檢查查詢語法
* 擷取表格說明

In [ ]:
from langchain.agents import create_sql_agent

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain_community.agent_toolkits import SQLDatabaseToolkit

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

### Agent 任務示範 #1 - 執行查詢


In [ ]:
agent_executor.run(
    "Which country's customers spent the most?"
)



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track I should query the Customer and Invoice tables to get the customer's country and the amount spent.
Action: sql_db_schema
Action Input: Customer, Invoice
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Em

'USA'

### Agent 任務範例 #2 - 描述表格與 Schema

In [ ]:
agent_executor.run("Describe the playlisttrack table")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track I should query the schema of the PlaylistTrack table.
Action: sql_db_schema
Action Input: PlaylistTrack
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/ I should query the first 10 rows from the PlaylistTrack table.
Action: sql_db_query
Action Input: SELECT * FROM PlaylistTrack LIMIT 10[(1, 3402), (1, 3389), (1, 3390), (1, 3391), (1, 3392), (1, 3393), (1, 3394), (1, 3395), (1, 3396), (1, 3397)] I now know the final answer
Final Answer: The PlaylistTrack table contains a list of tracks associated with each playlist

'The PlaylistTrack table contains a list of tracks associated with each playlist.'

### Agent 任務範例 #3 - 尋找並修正名詞拼寫錯誤

為了過濾包含專有名詞（例如地址、歌曲名稱或藝術家）的欄，我們首先需要仔細檢查拼寫，以便正確過濾數據。

我們可以通過使用數據庫中存在的所有不同的專有名詞來創建向量儲存來實現此目的。然後，我們可以讓 Agent 在使用者在問題中包含專有名詞時查詢該向量儲存，以找到該詞的正確拼寫。通過這種方式，Agent 可以確保在構建目標查詢之前了解使用者指的是哪個實體。

讓我們採用與 Few-shot 類似的做法，但沒有元數據：僅嵌入專有名詞，然後查詢以獲取與拼寫錯誤的使用者問題最相似的專有名詞。

首先，我們需要每個想要實體的唯一值，我們定義一個函數，將結果解析成元素清單：

In [ ]:
import ast
import re


def run_query_save_results(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return res


artists = run_query_save_results(db, "SELECT Name FROM Artist")
albums = run_query_save_results(db, "SELECT Title FROM Album")

現在，我們可以繼續建立自訂檢索工具和最終代理：

In [ ]:
!pip install --quiet faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 29.6 MB/s eta 0:00:00


In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

texts = artists + albums

embeddings = OpenAIEmbeddings()
vector_db = FAISS.from_texts(texts, embeddings)
retriever = vector_db.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="name_search",
    description="use to learn how a piece of data is actually written, can be from names, surnames addresses etc",
)

custom_tool_list = [retriever_tool]

In [ ]:
from langchain.agents import AgentType, create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI

# db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
If a user asks for me to filter based on proper nouns, I should first check the spelling using the name_search tool.
Otherwise, I can then look at the tables in the database to see what I can query.
Then I should query the schema of the most relevant tables
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
)

讓我們嘗試一下：

In [ ]:
agent.run("How many albums does alis in pains have?")



> Entering new SQL Agent Executor chain...

Invoking: `name_search` with `{'query': 'alis in pains'}`


House of Pain

Alice In Chains

Aisha Duo

House Of Pain
Invoking: `sql_db_list_tables` with `{}`
responded: The correct band name seems to be "Alice In Chains". Now I will check the tables in the database to see which one could contain the information about albums.

Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Album, Artist'}`
responded: The tables "Album" and "Artist" seem to be the most relevant for this query. I will now check their schemas to understand their structure and the relationships between them.


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Thos

'The band "Alice In Chains" has 1 album in the database.'

正如我們所見，代理使用了 `name_search` 工具來檢查如何正確地針對這個特定藝術家查詢資料庫。

### 想了解更多？

若要進一步了解 SQL 代理及其運作方式，我們參閱 [SQL Agent Toolkit](/docs/integrations/toolkits/sql_database)文件。

您也可以針對其他文件類型查看代理：
- [Pandas Agent](/docs/integrations/toolkits/pandas)
- [CSV Agent](/docs/integrations/toolkits/csv)